## Programación Lineal
### Coincidencia de Flujos de Caja
Si tenemos varias alternativas de financiación y disponemos de una planificación de los flujos de caja necesarios para un proyecto, podemos construir un problema de programación lineal para hacer coincidir los flujos de caja con el efectivo disponible en cada momento

In [2]:
import numpy as np
import pandas as pd
import cvxpy as cp

### Descripción 
Un proyecto de **5 años** tiene los siguientes flujos de caja estimados (en millones)

| Año | y1 | y2 | y3 | y4 | y5 |
|----:|---:|---:|---:|---:|---:|
|Flujos Netos | -5.2 | 0.4 | -1.5 | 2.7 | 7.8 |

Para el proyecto podemos financiarnos de la siguiente forma.
- Prestamos bancarios que se pagan al final de año a un interés del 0.03%
- Emitir bonos a 2 años con cupón anual del 0.02%
- El efectivo sobrante recibe un 0.005% anual invertido en mercados monetarios

### Modelo

In [3]:
# años
n=5

# flujos de caja
cf = np.array([-5.2, 0.4, -1.5, 2.7, 7.8 ])
cf

array([-5.2,  0.4, -1.5,  2.7,  7.8])

In [4]:
r_b = 0.03     #interes bancario
r_c = 0.02     #cupon bono
r_z = 0.005    #interes ahorro

In [5]:
X = cp.Variable(5) # cada año prestamos al banco
Y = cp.Variable(5) # cada año emision de bonos
Z = cp.Variable(5) # cada año efectivo sobrante 

In [6]:
objetivo = cp.Maximize(Z[4])

In [7]:
c_y1 =    X[0] + Y[0] - Z[0]                                         == -cf[0]
c_y2 =    X[1] + Y[1] - Z[1]  -X[0]*(1+r_b) -Y[0]*r_c + Z[0]*(1+r_z)   == -cf[1] 

In [8]:
constraints = [X >= 0, Y>=0, Z>=0]
constraints.append(c_y1)
constraints.append(c_y2)

In [9]:
for i in range(2,5):
    c_yi = X[i] + Y[i] - Z[i] - X[i-1]*(1+r_b) - Y[i-2]*(1+r_c) - Y[i-1]*r_c + Z[i-1]*(1+r_z) == -cf[i] 
    constraints.append(c_yi)

In [10]:
constraints.append(X[-1] == 0)
constraints.append(Y[-2:] == 0)

In [11]:
constraints

[Inequality(Constant(CONSTANT, ZERO, ())),
 Inequality(Constant(CONSTANT, ZERO, ())),
 Inequality(Constant(CONSTANT, ZERO, ())),
 Equality(Expression(AFFINE, UNKNOWN, ()), Constant(CONSTANT, NONNEGATIVE, ())),
 Equality(Expression(AFFINE, UNKNOWN, ()), Constant(CONSTANT, NONPOSITIVE, ())),
 Equality(Expression(AFFINE, UNKNOWN, ()), Constant(CONSTANT, NONNEGATIVE, ())),
 Equality(Expression(AFFINE, UNKNOWN, ()), Constant(CONSTANT, NONPOSITIVE, ())),
 Equality(Expression(AFFINE, UNKNOWN, ()), Constant(CONSTANT, NONPOSITIVE, ())),
 Equality(Expression(AFFINE, UNKNOWN, ()), Constant(CONSTANT, ZERO, ())),
 Equality(Expression(AFFINE, UNKNOWN, (2,)), Constant(CONSTANT, ZERO, ()))]

In [12]:
prob = cp.Problem(objetivo, constraints)

In [13]:
prob.solve(verbose=True)

                                     CVXPY                                     
                                    v1.1.15                                    
(CVXPY) Sep 18 01:17:00 PM: Your problem has 15 variables, 10 constraints, and 0 parameters.
(CVXPY) Sep 18 01:17:00 PM: It is compliant with the following grammars: DCP, DQCP
(CVXPY) Sep 18 01:17:00 PM: (If you need to solve this problem multiple times, but with different data, consider using parameters.)
(CVXPY) Sep 18 01:17:00 PM: CVXPY will first compile your problem; then, it will invoke a numerical solver to obtain a solution.
-------------------------------------------------------------------------------
                                  Compilation                                  
-------------------------------------------------------------------------------
(CVXPY) Sep 18 01:17:00 PM: Compiling problem (target solver=ECOS).
(CVXPY) Sep 18 01:17:00 PM: Reduction chain: FlipObjective -> Dcp2Cone -> CvxAttr2Constr -> Con

3.760172461482673

In [14]:
X.value.round(3)

array([0.293, 0.   , 2.544, 0.   , 0.   ])

In [15]:
Y.value.round(3)

array([4.907, 0.   , 3.961, 0.   , 0.   ])

In [16]:
Z.value.round(3)

array([0.  , 0.  , 0.  , 0.  , 3.76])

In [17]:
xs = np.round(X.value,3)
ys = np.round(Y.value,3)
zs = np.round(Z.value,3)

In [18]:
pd.DataFrame({
    'bancos':xs,
    'bonos':ys,
    'cash':zs,
    'cash_flow':cf
})

,bancos,bonos,cash,cash_flow
0,0.293,4.907,0.00,-5.2
1,0.000,0.000,0.00,0.4
2,2.544,3.961,0.00,-1.5
3,0.000,0.000,0.00,2.7
4,0.000,0.000,3.76,7.8
